# Распознование рукописных цифр (MNIST) с помощью Pytorch

### 1. Подключение библиотек

In [1]:
import torch
import torchvision
import torchvision.datasets
import torchvision.transforms
import torch.utils.data
import torch.nn
import os

### 2. Загрузка набора данных MNIST

__Скачивание данных в текущую директорию:__

In [2]:
dir_name = os.getcwd()

__Чтение тренировочной и тестовой выборок набора данных MNIST :__

Данные представляются в виде пар __(tuple)__, где первый элемент — изображение в формате __PIL.Image.Image__, а второй — целочисленная метка класса. Параметр __transform__ обеспечивает преобразование изображений в формат __torch.Tensor__ для последующей работы.

In [3]:
train_dataset = torchvision.datasets.MNIST(root = dir_name, train = True,
download = True, transform = torchvision.transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root = dir_name, train = False,
download = True, transform = torchvision.transforms.ToTensor())

__Зададим размер обрабатываемой пачки данных:__

In [4]:
batch_size = 1000

__Создадим объекты для последовательной загрузки пачек данных из тренировочной и тестовой выборок:__

In [5]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
batch_size = batch_size, shuffle = True)

test_data_loader = torch.utils.data.DataLoader(test_dataset,
batch_size = batch_size, shuffle = False)

### 3. Создание модели, соответствующей логистической регрессии

__Количество входных нейронов:__ 28 * 28 = 784, поскольку изображения имеют размер 28 на 28 пикселей

In [6]:
image_resolution = 28 * 28

__Количество выходных нейронов:__ 10, поскольку всего 10 классов (цифры от 0 до 9)

In [7]:
num_classes = 10

__Создадим класс сети, соответствующей логистической регрессии:__

In [8]:
class LogisticRegressionModel(torch.nn.Module):
    # Объявление конструктора
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        # Создание полносвязного слоя
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    # Переопределение метода, вызываемого в процессе прямого прохода
    def forward(self, x):
        out = self.linear(x)
        return out

__Создадим объект разработанного класса:__

In [9]:
logreg_nn = LogisticRegressionModel(image_resolution, num_classes)

### 3. Обучение построенной модели

__Зададим скорость обучения модели:__

In [10]:
learning_rate = 0.01

__Зададим количество эпох обучения модели:__

In [18]:
num_epochs = 100

__Выберем и зададим устройство для вычислений:__

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logreg_nn.to(device)
device

device(type='cpu')

__Выберем функцию ошибки на этапе обучения:__

In [20]:
loss_function = torch.nn.CrossEntropyLoss()

__Выберем метод оптимизации для обучения параметров:__

In [21]:
optimizer = torch.optim.SGD(logreg_nn.parameters(), lr = learning_rate)

__Обучим модель:__

In [22]:
%%time
for epoch in range(num_epochs): # проход по эпохам
    for i, (images, labels) in enumerate(train_data_loader): # проход по изображениям
        
        # Преобразование тензора [B, C, W, H] к формату [B, W * H]
        # (images.shape=[B, C, W, H], B - размер пачки, C = 1 - число каналов,
        # W, H - ширина и высота изображений в пачке)
        # и загрузка данных на устройство
        images = images.view(-1, image_resolution).requires_grad_().to(device)
        labels = labels.to(device)
        
        # Прямой проход
        outputs = logreg_nn(images) # вычисление выхода сети
        loss = loss_function(outputs, labels) # вычисление ошибки
        
        # Обратный проход
        optimizer.zero_grad() # обнуление всех вычисляемых градиентов
        loss.backward() # вычисление градиента функции ошибки
        optimizer.step() # обновление параметров модели

Wall time: 9min 42s


__Зададим функцию вычисления точности top-1:__

In [23]:
def get_accuracy(data_loader, model):
    tp = 0
    n = 0
    
    with torch.no_grad(): # деактивация вычисления градиентов
        for images, labels in data_loader: # проход по всем данным
            
            # Конвертация тензора [B, C, W, H] к формату [B, W * H]
            images = images.reshape(-1, image_resolution)
            
            outputs = model(images) # выход сети
            
            # Выбор предсказанных меток с максимальной достоверностью.
            # outputs.data - объект типа torch.tensor, двумерный тензор,
            # вектора достоверности принадлежности каждому из 10
            # допустимых классов (размерность 0 - номер изображения
            # в пачке, размерность 1 - номер класса); predicted - объект
            # типа torch.tensor (одномерный тензор меток классов).
            # Выбор максимальных значений по первой размерности
            _, predicted = torch.max(outputs.data, 1)
            
            n += labels.size(0) # количество изображений (= batch_size)
            
            tp += (predicted == labels).sum() # количество совпадений

    return tp / n

### 4. Тестирование обученной модели

__Логирование метрики качества на тренировочных данных:__

In [24]:
print('Test accuracy: {}'.format(get_accuracy(test_data_loader,logreg_nn)))

Test accuracy: 0.9045000076293945
